# Exercise - Data Modeling of Objects in Free Fall

<div class="alert alert-block alert-success">
This is an exercise about linear models. Their limitations will be identified and solutions based on non-linear models will be explored.


## <a id="sec:toc">Content</a>

[Learning Objectives](#sec_0)

[a) Data Exploration and Model Requirement Identification](#sec_a)

[b) Identify the Limitations of Linear Models](#sec_b)

[c) Explore a Non-Linear Mixture of Experts Models](#sec_c)

[d) Explore Locally Weighted Regression (LWR)](#sec_d)


### <a id="sec_0">Learning Objectives</a>

* Indicate relationships in data
* Use Error metrics (e.g., mean squared error) and understand their meaning
* Evaluate linear models and mixture models
* Apply locally weighted regression and analyze how the bandwidth parameter affects the model 

Import the needed libaries:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.spatial.distance import cdist
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import Axes3D

## <a id="sec:a">a) Data Exploration and Model Requirement Identification</a>

In the following part of this notebook we are going to have a look at our data and plot it. \
Try to find connections between the different physical quantities (fall time, height and weight).

In [ ]:
# DO NOT CHANGE

# Method to read a file into a dataset
def read_file_into_dataset(filename):
    dataset = []
    with open(filename, "r", encoding="utf-8") as file:  # Ensure the correct file extension
        # Read all lines, but skip the first line (the header)
        lines = file.readlines()[1:]  # Skip the first line (header)

        # Create the dataset from the remaining lines
        for line in lines:
            # Split the line by commas and convert values to float
            fall_time, weight, height = line.strip().split(",")
            dataset.append([float(fall_time), float(weight), float(height)])  # Append both values as a list of floats
    return dataset

# Scatter plot of Fall Time vs Height (m) or Fall Time vs Weight (g)
def plot_fall_time_vs_y(df, y_axis="Height (m)"):
    plt.figure()
    plt.scatter(df["Fall Time (s)"], df[y_axis], color='blue', marker='o', label="Measured Data")
    plt.grid(True)
    plt.title(f'Scatter Plot of Fall Time vs {y_axis}')
    plt.ylabel(y_axis)
    plt.xlabel('Fall Time (s)')
    plt.show()

def plot_fall_time_vs_height_weight(df):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Scatter plot for Fall Time vs Height vs Weight
    ax.scatter(df["Height (m)"], df["Weight (g)"], df["Fall Time (s)"], color='blue', marker='o', label="Measured Data")
    
    # Set labels and title
    ax.set_xlabel('Height (m)')
    ax.set_ylabel('Weight (g)')
    ax.set_zlabel('Fall Time (s)')
    ax.set_title('3D Scatter Plot of Fall Time vs Height and Weight')
    
    # Show legend
    ax.legend(loc="upper left")
    ax.view_init(elev=30, azim=140)
    
    plt.show()

def check_value(x, param_name):
    if x is None:
        print(f"x_axis is still None. Please enter a valid value for {param_name}.")
        raise Exception("Execution stopped because x_axis is not defined.")


### 1. Load & display the data set

In [ ]:
# Load the provided dataset containing measurements of falling times of objects with different weights from different heights
dataset = read_file_into_dataset("falltimes-weight-height-multiple-measurements.csv")

# Display the DataFrame in a tabular form
# Convert the dataset into a pandas DataFrame
df = pd.DataFrame(dataset, columns=["Fall Time (s)", "Weight (g)", "Height (m)"])
print(df)                          

### 2. Visualize scatter plot of Fall Time vs. Height or Weight

<u>TASK</u>: In the code cell below you can change the y_axis parameter to be able to better examine the relations between the variables.

In [ ]:
y_axis = None   # CHANGE y_axis variable from None to either "Height (m)" or "Weight (g)"
                # Include the quotation marks

# DO NOT CHANGE THE CODE BELOW
check_value(y_axis, "y_axis")

In [ ]:
# Visualize scatter plot of Fall Time vs. Height or Weight
plot_fall_time_vs_y(df, y_axis = y_axis)


### 3. Visualize 3D scatter plot

In [ ]:
# Visualize 3D scatter plot
plot_fall_time_vs_height_weight(df)

## <a id="sec:b">Identify the Limitations of Linear Models</a>

Now we will fit a linear model to the data. It will use the heights from which objects are dropped to try to predict the fall times. \
We will have a look at the predictions of this model in a plot. \
We will also compute the MSE of the predicted data. 
The mean squared error (MSE) is a metric that calculates \
the average of the squares of the differences between predicted and actual values, \
providing a measure of the accuracy of a model, with smaller values indicating better fit.

### Mean Squared Error (MSE)

$$
\text{MSE} = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2
$$


<div>
<img src="https://www.simplilearn.com/ice9/free_resources_article_thumb/Reg_Line.png" width="520"/>
</div>


In [ ]:
# DO NOT CHANGE

# Train linear regression model and return model coefficients
def train_linear_regression(df):
    lin_reg = LinearRegression()
    X = df["Height (m)"].values.reshape(-1, 1)  # Features (Weight)
    y = df["Fall Time (s)"]  # Target (Fall Time)

    # Split dataset into training and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the linear regression model
    lin_reg.fit(X_train, y_train)

    # Get the model parameters (c1, c2)
    print(f"Model Coefficients: c1 (bias) = {lin_reg.intercept_}, c2 (slope) = {lin_reg.coef_[0]}")

    # Return the trained model and the split data
    return lin_reg, X_test, y_test

# Calculate and print predicted and real accelerations
def calculate_and_print_accelerations(df, y_test, y_pred):
    # Use the test data weights and heights for this
    test_heights = df["Height (m)"][y_test.index].values  # Heights corresponding to the test set

    # Calculate predicted and real accelerations using a = h / t^2
    acceleration_pred = 2 * test_heights / (y_pred ** 2)
    acceleration_measured = 2 * test_heights / (y_test.values ** 2)

    print(f"Predicted Accelerations:\n {acceleration_pred}")
    print(f"Measured Accelerations:\n {acceleration_measured}")

# Visualize true vs. predicted fall times and model predictions
def visualize_predictions(df, y_test, y_pred, lin_reg):
    heights = df["Height (m)"][y_test.index]
    # Scatter plot: True vs. Predicted fall times (Fall Time vs. Height)
    plt.figure(num="True vs. Predicted fall times")
    plt.scatter(heights, y_test, color='blue', marker='o', label="True Measurements")
    plt.scatter(heights, y_pred, color='red', marker='x', label="Predicted Fall Times")
    plt.title("True vs. Predicted Fall Times (vs. Height)")
    plt.xlabel("Height (m)")
    plt.ylabel("Fall Time (s)")
    plt.grid(True)
    plt.legend()
    plt.show()


### 1. Train the linear regression model 

Training means that we try to adjust the parameters of our linear model so that it fits the data as well as possible. \
The linear model has the form:
$y = c_1*x + c_2$ \
We automatically choose $c_1$ & $c_2$ so that the line matches the data closely. So we want to achieve a low MSE. 

In [ ]:
# Train linear regression model
lin_reg, X_test, y_test = train_linear_regression(df)

### 2. Predict fall times for the test set and calculate accelerations

In [ ]:
# Predict fall times for the test set and calculate accelerations
y_pred = lin_reg.predict(X_test)
calculate_and_print_accelerations(df, y_test, y_pred)

### 3. Visualize predictions and linear model

In [ ]:
# Visualize predictions and linear model
visualize_predictions(df, y_test, y_pred, lin_reg)

### 4. Calculate the Mean Squared Error (MSE)

In [ ]:
# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE) between predicted and real fall times: {mse}")

## <a id="sec:c">c) Explore a Non-Linear Mixture of Expert Models</a>

In this chapter we will split our data into multiple subsets and train separate models for each of these subsets. \
The data will be split by height. We will also compute the total MSE to see if multiple models perform better than one.

In [ ]:
# DO NOT CHANGE

# Function to split data using a simple decision tree based on height
def split_data_by_height(df, num_splits):
    # Split the dataset into `num_splits` subsets based on height
    min_height = df["Height (m)"].min()
    max_height = df["Height (m)"].max()

    # Define height thresholds based on the number of splits
    thresholds = np.linspace(min_height, max_height, num_splits + 1)

    subsets = []
    for i in range(num_splits):
        if i < num_splits - 1:
            # Include lower bound and exclude upper bound for all but the last subset
            subset = df[(df["Height (m)"] >= thresholds[i]) & (df["Height (m)"] < thresholds[i + 1])]
        else:
            # Include both bounds for the last subset
            subset = df[(df["Height (m)"] >= thresholds[i]) & (df["Height (m)"] <= thresholds[i + 1])]
        subsets.append(subset)

    return subsets, thresholds


# Function to train linear models with train-test splits for each subset of the data
def train_linear_models_with_split(subsets, test_size=0.2, random_state=42):
    models = []
    subset_data = []  # To store the train and test sets for each subset

    for subset in subsets:
        X = subset["Height (m)"].values.reshape(-1, 1)
        y = subset["Fall Time (s)"]

        # Perform train-test split for each subset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Train the linear regression model on the training data
        lin_reg = LinearRegression()
        lin_reg.fit(X_train, y_train)

        # Store the model and train-test data
        heights_test = subset.loc[y_test.index, "Height (m)"]  # Ensure heights correspond only to the test data
        models.append(lin_reg)
        subset_data.append({
            "X_train": X_train, "X_test": X_test,
            "y_train": y_train, "y_test": y_test,
            "heights_test": heights_test.values  # Store only the heights corresponding to the test set
        })

    return models, subset_data


# Function to compute predictions and errors for the test sets of each subset
def compute_predictions_and_errors_with_split(subset_data, models):
    total_weighted_mse = 0
    total_data_points = 0

    # Define reversed viridis colormap for actual values and inferno for predicted values
    actual_colors = np.flip(plt.cm.viridis(np.linspace(0, 1, len(subset_data))), axis=0)
    predicted_colors = plt.cm.inferno(np.linspace(0, 1, len(subset_data)))

    for i, data in enumerate(subset_data):
        X_test = data["X_test"]
        y_test = data["y_test"]
        heights_test = data["heights_test"]

        # Predict the fall times using the trained model
        y_pred = models[i].predict(X_test)

        # Calculate the Mean Squared Error (MSE) for the test set
        mse = mean_squared_error(y_test, y_pred)
        n_data_points = len(y_test)  # Number of data points in this subset's test set

        # Accumulate weighted MSE: n_data_points * mse
        total_weighted_mse += n_data_points * mse
        total_data_points += n_data_points  # Keep track of total number of data points

        print(f"Subset {i + 1} MSE: {mse}")

        # Plot the true and predicted values for each subset (Fall Time vs. Height)
        plt.figure()
        plt.scatter(heights_test, y_test, color=actual_colors[i], marker='o', 
                    label=f"True Data (Subset {i + 1})", alpha=0.8)
        plt.scatter(heights_test, y_pred, color=predicted_colors[i], marker='x', 
                    label=f"Predicted Data (Subset {i + 1})", alpha=0.8)
        plt.title(f"Subset {i + 1}: True vs. Predicted Fall Times (Test Set)")
        plt.xlabel("Height (m)")
        plt.ylabel("Fall Time (s)")
        plt.grid(True)
        plt.legend(loc="lower right", fontsize=9)
        plt.ylim(-0.5, 5.1)
        plt.xlim(-0.5, 101)
        plt.show()

    # Calculate the combined MSE using the total weighted MSE and total number of data points
    combined_mse = total_weighted_mse / total_data_points
    print(f"Total (Weighted) MSE across all subsets: {combined_mse}")

# Function to plot Fall Time vs Height (True vs Predicted)
def plot_falltime_vs_height(subset_data, models):
    plt.figure(figsize=(10, 6))

    # Define reversed viridis colormap for actual values and inferno for predicted values
    actual_colors = np.flip(plt.cm.viridis(np.linspace(0, 1, len(subset_data))), axis=0)
    predicted_colors = plt.cm.inferno(np.linspace(0, 1, len(subset_data)))

    for i, data in enumerate(subset_data):
        heights_test = data["heights_test"]  # Heights corresponding only to the test set
        X_test = data["X_test"]
        y_test = data["y_test"]

        # Predict the fall times using the trained model
        y_pred = models[i].predict(X_test)
        
        # Plot true fall times (reversed viridis colormap)
        plt.scatter(heights_test, y_test, color=actual_colors[i], marker='o', 
                    label=f"True Data (Subset {i + 1})", alpha=0.8)
        
        # Plot predicted fall times (inferno colormap)
        plt.scatter(heights_test, y_pred, color=predicted_colors[i], marker='x', 
                    label=f"Predicted Data (Subset {i + 1})", alpha=0.8)

    plt.title("True vs Predicted Fall Times for All Models")
    plt.xlabel("Height (m)")
    plt.ylabel("Fall Time (s)")
    plt.grid(True)
    plt.legend(loc="best", fontsize=9)
    plt.ylim(-0.5, 5.1)
    plt.xlim(-0.5, 101)
    plt.show()

    


### 1. Split data by height

<u>TASK:</u> In the code cell below you can decide the number of subsets by changing the parameter ``num_splits``.

In [ ]:
num_splits = None # CHANGE num_splits variable from None to natural numbers in the interval [1, 34]

# DO NOT CHANGE THE CODE BELOW
check_value(num_splits, "num_splits")
# Split data by height using the decision tree with a specified number of splits
subsets, thresholds = split_data_by_height(df, num_splits)

### 2. Train individual linear models for each subset

In [ ]:
# Train individual linear models for each subset with train-test splits
models, subset_data = train_linear_models_with_split(subsets)

### 3. Compute predictions and errors for each test set of the subsets

In [ ]:
# Compute predictions and errors for each test set of the subsets
compute_predictions_and_errors_with_split(subset_data, models)

### 4. Plot Fall Time vs Height (True vs Predicted) for all models

In [ ]:
# Plot Fall Time vs Height (True vs Predicted) for all models
plot_falltime_vs_height(subset_data, models)

## <a id="sec:c">Explore Locally Weighted Regression (LWR)</a>

In this part of the notebook we will make predictions using a locally weighted regression \
model for one datapoint of each of the previously defined subsets. \
Locally weighted regression fits a model to a target point by giving nearby data \
points more influence than those further away. \
In this example of LWR these distances between the training data points and the test data point \
are computed using the heights from which our objects fall. \
The "weights" are computed using a gaussian normal distribution centered at the target point \
(the data for which we are trying to make a prediction). \
The "weights" are a measure for how strong a training data point will influence the prediction.\
Do not mistake these weights with the masses of the objects.


In [ ]:
# DO NOT CHANGE

# Function to compute distances and importance weights for test samples (only based on training weights)
def compute_distances_and_weights(train, test, bandwidth=1.0):
    # Extract height values for distance computation (ignore weight)
    train_weights = train[["Height (m)"]].values
    test_weights = test[["Height (m)"]].values

    # Compute Euclidean distances between test and training weights
    distances = cdist(test_weights, train_weights, metric='euclidean').flatten()

    # Compute Gaussian kernel weights based on the distances
    weights = np.exp(-distances ** 2 / (2 * bandwidth ** 2))

    # Normalize the weights so they sum to 1
    weights = weights / np.sum(weights)

    return distances, weights


# Function to predict fall time using weighted linear regression (based only on weights and fall times)
def weighted_linear_regression(train, test, weights):
    # Prepare the training data (use weights and fall times)
    X_train = train[["Height (m)"]].values
    y_train = train["Fall Time (s)"].values

    # Add a column of ones to include the bias term
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]  # Now X_train is N x 2

    # Compute the weighted linear regression using the normal equation
    W = np.diag(weights)  # Diagonal weight matrix
    XTX = X_train.T @ W @ X_train  # Now XTX is 2 x 2
    XTy = X_train.T @ W @ y_train  # Now XTy is 2 x 1

    # Solve for theta (linear regression coefficients)
    theta = np.linalg.inv(XTX) @ XTy
    #print("theta =", theta)

    # Prepare test data (add bias term)
    X_test = test[["Height (m)"]].values
    X_test = np.c_[np.ones(X_test.shape[0]), X_test]  # Now X_test is M x 2

    # Predict the fall time for the test data
    y_pred = X_test @ theta

    return y_pred


def visualize_all_without_predictions(df, num_splits, seed=None):
    plt.figure(figsize=(10, 6))

    # Split the data into subsets based on height (for splitting only, height is ignored afterward)
    subsets, thresholds = split_data_by_height(df, num_splits)

    # Dictionary to store the selected test samples for later use
    selected_test_samples = {}

    # Define a color map for the subsets to ensure distinct colors
    colors = plt.cm.viridis(np.linspace(0, 1, num_splits))

    # Iterate over each subset (each expert) and visualize without predictions
    for i, subset in enumerate(subsets):
        if subset.empty:
            continue  # Skip empty subsets

        # Extract as many samples as the subset index (i + 1) and sort them by index
        extracted_samples = subset.sample(n=(i + 1), random_state=seed)
    
        # Always select the i-th sample as the test sample (0-based index)
        test_sample = extracted_samples.iloc[[i]]
        training_samples = subset.drop(test_sample.index)  # Use the rest as training data

        # Store the test sample for later use
        selected_test_samples[i] = test_sample

        # Plot the training data points (fall times vs heights)
        plt.scatter(training_samples["Height (m)"], training_samples["Fall Time (s)"], 
                    color=colors[i], label=f"Training Data (Subset {i + 1})")

        # Plot the test data points (fall times vs heights)
        plt.scatter(test_sample["Height (m)"], test_sample["Fall Time (s)"], marker='x',
                    color='green', label=f"Test Data (Subset {i + 1})", s=150)
    
    plt.title(f"Training and Test Fall Times (Fall Time vs Height)")
    plt.xlabel("Height (m)")
    plt.ylabel("Fall Time (s)")
    plt.legend(loc="best")
    plt.grid(True)
    plt.show()


    # Return the selected test samples
    return selected_test_samples

# Function to compute MSE and print it
def compute_mse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    return mse

# Function to visualize all training, test data, and predictions in one plot, with MSE calculation
def visualize_all_with_predictions(df, num_splits, selected_test_samples, bandwidth=1.0):
    plt.figure(figsize=(10, 6))

    # Define a color map for the subsets to ensure distinct colors
    colors = plt.cm.viridis(np.linspace(0, 1, num_splits))

    # Split the data into subsets based on height (for splitting only, height is ignored afterward)
    subsets, thresholds = split_data_by_height(df, num_splits)

    # Initialize list to store actual and predicted values for MSE computation across all subsets
    actual_values = []
    predicted_values = []

    # Iterate over each subset (each expert) and visualize with predictions
    for i, subset in enumerate(subsets):
        if subset.empty:
            continue  # Skip empty subsets

        # Retrieve the test sample that was used in the first visualization
        test_sample = selected_test_samples.get(i)
        if test_sample is None:
            continue  # Skip if no test sample was recorded

        # Use the entire dataset (excluding the test sample itself) as training data
        training_samples = df.drop(test_sample.index)  # All data except the selected test sample

        # Compute distances and weights (based on the entire dataset, excluding the test sample itself)
        distances, weights = compute_distances_and_weights(training_samples, test_sample, bandwidth=bandwidth)

        # Predict the fall time using weighted linear regression (based only on weights and fall times)
        y_pred = weighted_linear_regression(training_samples, test_sample, weights)

        # Store the actual and predicted values for MSE calculation
        actual_values.extend(test_sample["Fall Time (s)"])
        predicted_values.extend(y_pred)

        # Plot the training data points from the current subset (fall times vs heights)
        plt.scatter(subset["Height (m)"], subset["Fall Time (s)"], color=colors[i], label=f"Training Data (Subset {i + 1})", alpha=0.6)

        # Plot the test data points (fall times vs heights)
        plt.scatter(test_sample["Height (m)"], test_sample["Fall Time (s)"], marker='x', color='green', label=f"Test Data (Subset {i + 1})", s=150)

        # Plot the predicted fall times (using the same heights as the test sample)
        plt.scatter(test_sample["Height (m)"], y_pred, marker='x', color='red', label=f"Predicted Data (Subset {i + 1})", s=150)

    # Calculate and display the overall MSE across all subsets
    plt.title(f"Training, Test, and Predicted Fall Times (Fall Time vs Height)")
    plt.xlabel("Height (m)")
    plt.ylabel("Fall Time (s)")
    plt.legend(loc="best")
    plt.grid(True)
    plt.show()
    overall_mse = compute_mse(actual_values, predicted_values)


### 1. Visualize test and training data

<u>TASK:</u> In the code cell below you can decide the number of subsets used in this section by changing the parameter ``num_splits``.

In [ ]:
num_splits = None # CHANGE num_splits variable from None to natural numbers in the interval [1, 21]

# DO NOT CHANGE THE CODE BELOW
check_value(num_splits, "num_splits")

In [ ]:
# Select test and training data and visualize it in a scatter plot
selected_samples = visualize_all_without_predictions(df, num_splits, seed=43)

<u>TASK:</u> In the code cell below you can change the bandwidth parameter to increase or decrease the dependency \
of the predictions on the distances of other data samples.

In [ ]:
bandwidth = None # CHANGE bandwidth variable from None to rational numbers in the interval [0, 100]

# DO NOT CHANGE THE CODE BELOW
check_value(bandwidth, "bandwidth")

### 2. Visualize predictions with the specified bandwidth parameter

In [ ]:
# Compute the distances of training data to test data points and the importance weight matrix, visualize the predictions
visualize_all_with_predictions(df, num_splits, selected_samples, bandwidth = bandwidth) 